In [1]:
# Import Libraries
import pandas as pd
import numpy as np
from keras.optimizers import Adam
from keras.models import load_model
from joblib import load
from keras import backend as K

In [2]:
# Load the saved model
loaded_ann_model = load_model('F:/Works/Application of ML/Final/ANN_trained_model.h5')

In [3]:
# Compile the loaded ANN model with the custom optimizer
loaded_ann_model.compile(optimizer=Adam(learning_rate=0.0001 ), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [4]:
#mse function
def mse(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

In [5]:
# Load the saved model with custom objects
loaded_lstm_model = load_model('F:/Works/Application of ML/Final/LSTM_trained_model.h5', custom_objects={'mse': mse})

In [6]:
# Load the scaler for LSTM
scaler_LSTM = load('F:/Works/Application of ML/Final/scaler_LSTM.joblib')

# Load the scaler for ANN
scaler_ANN = load('F:/Works/Application of ML/Final/scaler_ANN.joblib')

In [7]:
# Load the original data used during LSTM Model training
LSTM_cleaned_data = pd.read_csv('F:/Works/Application of ML/Final/LSTM_cleaned_data.csv')  # Load your original training data here
LSTM_cleaned_data

,PM2.5,PM10,SO2,NO2,CO,O3
0,16.665354,41.316808,2.818773,16.060654,0.430841,36.267782
1,17.288915,39.965232,2.423315,14.354335,0.430870,27.113356
2,9.280876,21.127849,1.855501,13.340066,0.410151,20.977514
3,22.639594,53.866619,2.739353,12.817808,0.403904,33.445877
4,24.576782,59.886589,3.123391,17.698004,0.488798,32.663070
...,...,...,...,...,...,...
2705,9.504518,15.533632,0.965425,13.304156,0.385762,29.021385
2706,11.450000,22.141246,1.118793,10.305741,0.343775,35.827846
2707,10.667974,18.257115,0.998958,9.099509,0.331951,37.641503
2708,12.035394,23.145436,1.186963,10.338576,0.314859,31.804717


In [8]:
# Fit the scaler with the original data
scaler_LSTM.fit(LSTM_cleaned_data)

MinMaxScaler()

In [9]:
# Define the range of dates for which you want to make predictions
start_date = pd.to_datetime('2024-06-01')
end_date = pd.to_datetime('2024-06-30')

In [10]:
# Define the window size and features
window_size = 10  # Adjust this based on your model architecture
num_features = 6  # Adjust this based on the number of features in your input data

# Initialize the input data with dummy values (you should replace this with your actual data)
input_data = np.random.rand(window_size, num_features)

# Create a sequence of future dates within the specified range
future_dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Create an empty array to store the predicted values
predicted_values = []

# Predict the next value for each day in the specified range
for _ in range(len(future_dates)):
    # Reshape the input data to match the model input shape
    input_data_reshaped = np.reshape(input_data, (1, input_data.shape[0], input_data.shape[1]))
    
    # Predict the next value using the loaded LSTM model
    next_value = loaded_lstm_model.predict(input_data_reshaped)[0]
    
    # Append the predicted value to the list of predicted values
    predicted_values.append(next_value)
    
    # Update the input data by removing the first time step and appending the predicted value
    input_data = np.append(input_data[1:], [next_value], axis=0)

# Inverse transform the predicted values to get the original scale
predicted_values = scaler_LSTM.inverse_transform(predicted_values)

# Create a DataFrame to store the predicted values along with corresponding dates
predicted_df = pd.DataFrame(predicted_values, columns=['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3'])
predicted_df['Date'] = future_dates

# Display the predicted DataFrame
predicted_df

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━

,PM2.5,PM10,SO2,NO2,CO,O3,Date
0,23.148675,41.118336,2.554996,12.878099,-0.011401,33.178160,2024-06-01
1,20.779917,38.938098,2.430920,12.396253,-0.045915,31.873576,2024-06-02
2,19.188955,37.455901,2.360494,11.946347,-0.076783,30.879323,2024-06-03
3,18.146506,36.350566,2.271686,11.583330,-0.098255,30.045331,2024-06-04
4,17.290149,35.225492,2.215635,11.269204,-0.120491,29.359741,2024-06-05
5,16.171485,33.845775,2.168670,10.767349,-0.161026,28.669671,2024-06-06
6,15.281197,32.497133,2.072444,10.330281,-0.189650,28.084421,2024-06-07
7,14.425257,30.858027,1.992574,9.904931,-0.227252,27.285189,2024-06-08
8,13.782310,29.602322,1.910461,9.566256,-0.261873,26.485391,2024-06-09
9,13.038591,28.027477,1.828076,9.235325,-0.290791,25.877904,2024-06-10


In [11]:
# PM2.5 Sub-Index calculation
def get_PM25_subindex(x):
    if x <= 30:
        return x * 50 / 30
    elif x <= 60:
        return 50 + (x - 30) * 50 / 30
    elif x <= 90:
        return 100 + (x - 60) * 100 / 30
    elif x <= 120:
        return 200 + (x - 90) * 100 / 30
    elif x <= 250:
        return 300 + (x - 120) * 100 / 130
    elif x > 250:
        return 400 + (x - 250) * 100 / 130
    else:
        return 0

predicted_df["PM2.5_SubIndex"] = predicted_df["PM2.5"].apply(lambda x: get_PM25_subindex(x))

In [12]:
# PM10 Sub-Index calculation
def get_PM10_subindex(x):
    if x <= 50:
        return x
    elif x <= 100:
        return x
    elif x <= 250:
        return 100 + (x - 100) * 100 / 150
    elif x <= 350:
        return 200 + (x - 250)
    elif x <= 430:
        return 300 + (x - 350) * 100 / 80
    elif x > 430:
        return 400 + (x - 430) * 100 / 80
    else:
        return 0

predicted_df["PM10_SubIndex"] = predicted_df["PM10"].apply(lambda x: get_PM10_subindex(x))

In [13]:
# SO2 Sub-Index calculation
def get_SO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 380:
        return 100 + (x - 80) * 100 / 300
    elif x <= 800:
        return 200 + (x - 380) * 100 / 420
    elif x <= 1600:
        return 300 + (x - 800) * 100 / 800
    elif x > 1600:
        return 400 + (x - 1600) * 100 / 800
    else:
        return 0

predicted_df["SO2_SubIndex"] = predicted_df["SO2"].apply(lambda x: get_SO2_subindex(x))

In [14]:
# NO2 Sub-Index calculation
def get_NO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 180:
        return 100 + (x - 80) * 100 / 100
    elif x <= 280:
        return 200 + (x - 180) * 100 / 100
    elif x <= 400:
        return 300 + (x - 280) * 100 / 120
    elif x > 400:
        return 400 + (x - 400) * 100 / 120
    else:
        return 0

predicted_df["NO2_SubIndex"] = predicted_df["NO2"].apply(lambda x: get_NO2_subindex(x))

In [15]:
# CO Sub-Index calculation
def get_CO_subindex(x):
    if x <= 1:
        return x * 50 / 1
    elif x <= 2:
        return 50 + (x - 1) * 50 / 1
    elif x <= 10:
        return 100 + (x - 2) * 100 / 8
    elif x <= 17:
        return 200 + (x - 10) * 100 / 7
    elif x <= 34:
        return 300 + (x - 17) * 100 / 17
    elif x > 34:
        return 400 + (x - 34) * 100 / 17
    else:
        return 0

predicted_df["CO_SubIndex"] = predicted_df["CO"].apply(lambda x: get_CO_subindex(x))

In [16]:
# O3 Sub-Index calculation
def get_O3_subindex(x):
    if x <= 50:
        return x * 50 / 50
    elif x <= 100:
        return 50 + (x - 50) * 50 / 50
    elif x <= 168:
        return 100 + (x - 100) * 100 / 68
    elif x <= 208:
        return 200 + (x - 168) * 100 / 40
    elif x <= 748:
        return 300 + (x - 208) * 100 / 539
    elif x > 748:
        return 400 + (x - 400) * 100 / 539
    else:
        return 0

predicted_df["O3_SubIndex"] = predicted_df["O3"].apply(lambda x: get_O3_subindex(x))

In [17]:
predicted_df.head()

,PM2.5,PM10,SO2,NO2,CO,O3,Date,PM2.5_SubIndex,PM10_SubIndex,SO2_SubIndex,NO2_SubIndex,CO_SubIndex,O3_SubIndex
0,23.148675,41.118336,2.554996,12.878099,-0.011401,33.178160,2024-06-01,38.581125,41.118336,3.193744,16.097623,-0.570041,33.178160
1,20.779917,38.938098,2.430920,12.396253,-0.045915,31.873576,2024-06-02,34.633195,38.938098,3.038650,15.495317,-2.295774,31.873576
2,19.188955,37.455901,2.360494,11.946347,-0.076783,30.879323,2024-06-03,31.981591,37.455901,2.950617,14.932934,-3.839153,30.879323
3,18.146506,36.350566,2.271686,11.583330,-0.098255,30.045331,2024-06-04,30.244177,36.350566,2.839608,14.479162,-4.912755,30.045331
4,17.290149,35.225492,2.215635,11.269204,-0.120491,29.359741,2024-06-05,28.816915,35.225492,2.769543,14.086504,-6.024531,29.359741


In [18]:
# Assuming 'df' is your DataFrame
real_data = predicted_df.iloc[:, -6:]
real_data.head()

,PM2.5_SubIndex,PM10_SubIndex,SO2_SubIndex,NO2_SubIndex,CO_SubIndex,O3_SubIndex
0,38.581125,41.118336,3.193744,16.097623,-0.570041,33.178160
1,34.633195,38.938098,3.038650,15.495317,-2.295774,31.873576
2,31.981591,37.455901,2.950617,14.932934,-3.839153,30.879323
3,30.244177,36.350566,2.839608,14.479162,-4.912755,30.045331
4,28.816915,35.225492,2.769543,14.086504,-6.024531,29.359741


In [19]:
# Transform the new data using the loaded scaler
real_data_scaled = scaler_ANN.transform(real_data)
real_data_scaled

array([[-0.72717337, -0.89574992, -0.96579565, -0.79851509, -1.02278554,
        -0.51052477],
       [-0.76698742, -0.92406345, -0.97828504, -0.82104336, -1.04865673,
        -0.54438239],
       [-0.7937283 , -0.94331192, -0.98537413, -0.84207832, -1.07179416,
        -0.5701861 ],
       [-0.81124976, -0.95766629, -0.99431348, -0.85905089, -1.08788898,
        -0.59183055],
       [-0.8256434 , -0.972277  , -0.9999556 , -0.87373759, -1.10455606,
        -0.60962354],
       [-0.84444589, -0.99019462, -1.00468301, -0.89720135, -1.13494001,
        -0.62753281],
       [-0.85940984, -1.00770869, -1.01436909, -0.91763609, -1.15639563,
        -0.6427217 ],
       [-0.87379648, -1.02899484, -1.02240879, -0.93752294, -1.18458135,
        -0.66346403],
       [-0.88460311, -1.04530198, -1.03067432, -0.95335741, -1.2105318 ,
        -0.68422108],
       [-0.89710354, -1.06575361, -1.03896711, -0.96882978, -1.23220848,
        -0.69998709],
       [-0.90776548, -1.08216974, -1.04491896, -0.

In [20]:
# Make predictions using the loaded model
predictions = loaded_ann_model.predict(real_data_scaled)
predictions

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step


array([[9.79317546e-01, 2.06824280e-02, 4.87599863e-08, 5.82984487e-15,
        1.22784346e-14, 3.76732629e-12],
       [9.89334643e-01, 1.06653031e-02, 1.26477948e-08, 9.34716527e-16,
        2.74108756e-15, 1.39232723e-12],
       [9.92156506e-01, 7.84346275e-03, 6.30755093e-09, 3.55213934e-16,
        1.24165967e-15, 8.00879111e-13],
       [9.93403614e-01, 6.59640133e-03, 4.03155109e-09, 1.81876132e-16,
        7.07145244e-16, 5.34248535e-13],
       [9.94285405e-01, 5.71458973e-03, 2.76108336e-09, 1.02211658e-16,
        4.31125715e-16, 3.75116686e-13],
       [9.94924545e-01, 5.07539744e-03, 1.85097204e-09, 5.47127697e-17,
        2.54458491e-16, 2.58808383e-13],
       [9.95498180e-01, 4.50181635e-03, 1.28183308e-09, 3.04291425e-17,
        1.53033707e-16, 1.80395073e-13],
       [9.95893598e-01, 4.10642009e-03, 9.27601662e-10, 1.77388154e-17,
        9.46658845e-17, 1.28820348e-13],
       [9.96075213e-01, 3.92471720e-03, 7.41149253e-10, 1.20563617e-17,
        6.73525527e-17, 

In [21]:
# Get the index of the class with the highest probability for each sample
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [22]:
# Define a dictionary mapping encoded numbers to original class labels
class_mappings = {
    0: 'Good',
    1: 'Satisfactory',
    2: 'Moderate',
    3: 'Poor',
    4: 'Very Poor',
    5: 'Severe'
}

# Replace the encoded numbers with their original class labels
predicted_class_labels = [class_mappings[label] for label in predicted_labels]

# Create a DataFrame to display the predicted class labels
predicted_df_ann = pd.DataFrame({'AQI Prediction': predicted_class_labels})
predicted_df_ann

,AQI Prediction
0,Good
1,Good
2,Good
3,Good
4,Good
5,Good
6,Good
7,Good
8,Good
9,Good


In [23]:
# Show in dataframe
first_six_columns_df = predicted_df.iloc[:, :7]
result_df = pd.concat([first_six_columns_df, predicted_df_ann], axis=1)
result_df

,PM2.5,PM10,SO2,NO2,CO,O3,Date,AQI Prediction
0,23.148675,41.118336,2.554996,12.878099,-0.011401,33.178160,2024-06-01,Good
1,20.779917,38.938098,2.430920,12.396253,-0.045915,31.873576,2024-06-02,Good
2,19.188955,37.455901,2.360494,11.946347,-0.076783,30.879323,2024-06-03,Good
3,18.146506,36.350566,2.271686,11.583330,-0.098255,30.045331,2024-06-04,Good
4,17.290149,35.225492,2.215635,11.269204,-0.120491,29.359741,2024-06-05,Good
5,16.171485,33.845775,2.168670,10.767349,-0.161026,28.669671,2024-06-06,Good
6,15.281197,32.497133,2.072444,10.330281,-0.189650,28.084421,2024-06-07,Good
7,14.425257,30.858027,1.992574,9.904931,-0.227252,27.285189,2024-06-08,Good
8,13.782310,29.602322,1.910461,9.566256,-0.261873,26.485391,2024-06-09,Good
9,13.038591,28.027477,1.828076,9.235325,-0.290791,25.877904,2024-06-10,Good


In [24]:
# Reorder columns with "Date" at the beginning
result_df = result_df[['Date', 'PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'AQI Prediction']]


In [25]:
# Round numeric values to 2 decimal places
numeric_columns = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']
result_df[numeric_columns] = result_df[numeric_columns].round(2)
result_df

,Date,PM2.5,PM10,SO2,NO2,CO,O3,AQI Prediction
0,2024-06-01,23.15,41.12,2.55,12.88,-0.01,33.18,Good
1,2024-06-02,20.78,38.94,2.43,12.40,-0.05,31.87,Good
2,2024-06-03,19.19,37.46,2.36,11.95,-0.08,30.88,Good
3,2024-06-04,18.15,36.35,2.27,11.58,-0.10,30.05,Good
4,2024-06-05,17.29,35.23,2.22,11.27,-0.12,29.36,Good
5,2024-06-06,16.17,33.85,2.17,10.77,-0.16,28.67,Good
6,2024-06-07,15.28,32.50,2.07,10.33,-0.19,28.08,Good
7,2024-06-08,14.43,30.86,1.99,9.90,-0.23,27.29,Good
8,2024-06-09,13.78,29.60,1.91,9.57,-0.26,26.49,Good
9,2024-06-10,13.04,28.03,1.83,9.24,-0.29,25.88,Good


In [26]:
# Export DataFrame to a CSV file
result_df.to_csv('F:/Works/Application of ML/Final/prediction_ANN.csv', index=False)